## CS4243, Week 12 Lab Plan
### 2023/24- S1
### Part 1: Ask Your Mini project-related questions, 


## 
### Part 2: Ask Your CW3-related questions, 


## 

## Part 3: Object Detection Using Yolo v3
### Amir - 2023
#### A few good resources: https://pjreddie.com/  ,  https://www.datacamp.com/blog/yolo-object-detection-explained  

### Download the YOLO pre-trained weights and configuration file:

- YOLO has several versions (YOLOv1, YOLOv2, YOLOv3, etc.).
- We focus on Yolo v3 
- you can find YOLOv3 weights and configuration files (yolov3.cfg and yolov3.weights) in Canvas/ Files/ Set6 folder and place them in your project directory.
- You can use OpenCV to load the YOLO model using the cv2.dnn.readNet function. Make sure you provide the paths to the configuration and weights files you downloaded earlier. 
- dnn stands for deep neural network. openCV employs that as a class for deep learning model. 

In [17]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("./yolo/yolov3.cfg", "./yolo/yolov3.weights")


Now net cotains a trained Yolo v3 model
YOLO is trained on the COCO (Common Objects in Context) dataset, s
o you'll need to load the class labels to map the class IDs to human-readable labels. 
You can download the labels again from Canvas/ Files/ Set6 folder
Although they are all available on the web too.
COCO contains: 

####  person , bicycle, car, motorbike, aeroplane, bus, train, truck, boat, bench, cat, dog, horse, sheep, cow, ....
would be saved in variable classes

In [18]:
with open("./yolo/coco.names", "r") as f:
    classes = f.read().strip().split("\n")


#### Now you can try any image to see how Yolo works. 
#### go for images with a few known, medium-size objects inside 
### images to try (test image):
1. still_life_light_photo.jpg 
2. 16335.jpg 
3. offset_1094232.jpg
4. IMG_20211204_123437.jpg
5. 0841b64699e5f9d1c1bb628b11a50151.jpg 

all avialble at Canvas, same directory. 

#### 3 important variable that are going to be generated here:
1. blob: generated by dnn.blobFromImage , contains the blobs detected in the test image. blob will be the input of our net
2. layer_names which is the unconnected outlayers of the net
3. detections: answers generated by your net when the input is the test image 

In [25]:
image = cv2.imread("./yolo/IMG_7471.JPG")
# image = cv2.imread("../../../Downloads/0200869_220904_threat_6503_240.png")
height, width, _ = image.shape
print(height, width)
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

layer_names = net.getUnconnectedOutLayersNames()
detections = net.forward(layer_names)


2592 3888


: 

#### Let's see what we have got in detections

In [20]:
print( type(detections) , len(detections) , len(detections[0]))
print(detections)

<class 'tuple'> 3 507
(array([[0.03899167, 0.05419095, 0.37402165, ..., 0.        , 0.        ,
        0.        ],
       [0.04278925, 0.03642969, 0.29820463, ..., 0.        , 0.        ,
        0.        ],
       [0.04677168, 0.04191811, 0.8526776 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.96287316, 0.9529423 , 0.38496146, ..., 0.        , 0.        ,
        0.        ],
       [0.96774906, 0.96623874, 0.33863387, ..., 0.        , 0.        ,
        0.        ],
       [0.964474  , 0.9641712 , 0.83607894, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.01910068, 0.02556489, 0.05088575, ..., 0.        , 0.        ,
        0.        ],
       [0.01888695, 0.01664926, 0.41781428, ..., 0.        , 0.        ,
        0.        ],
       [0.02112211, 0.01654611, 0.07129858, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97384554, 0.9773058 , 0.05041542, ..., 0.        , 0.        ,
        0.       

#### Process the detections:
You can iterate through the detections and filter out objects with confidence 
scores above a certain threshold. Then, you can draw bounding boxes and labels on the image to 
visualize the detected objects.

- detections is  3d array, contains detected objects features
- detection is a row of detections, means a detected object
- object is the features of the detected object

#### detections: (N, N\*grid_height\*grid_width, 85), where:

- N: The number of anchor boxes per grid cell. YOLOv3 uses three anchor boxes by default, so N is 3.
- 85: Each detection is represented by 85 values, which include:
- - 4 values for the bounding box coordinates (center_x, center_y, width, height).
- - 1 value for the objectness score (how likely it is an object).
- - 80 values for class probabilities (there are 80 classes in the COCO dataset). maximum shows to most probabel class of the object

- grid_height and grid_width: These represent the grid dimensions used for dividing the input image. YOLO divides the image into a grid and performs object detection within each grid cell. Usually it is 13x13 or so.

In [21]:
for detection in detections:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5:  # You can adjust the confidence threshold
            center_x = int(obj[0] * width)
            center_y = int(obj[1] * height)
            w = int(obj[2] * width)
            h = int(obj[3] * height)

            # Calculate the coordinates of the bounding box
            x = int(center_x - w/2)
            y = int(center_y - h/2)

            # Draw bounding box and label
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            label = f"{classes[class_id]}: {confidence:.2f}"
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [22]:
cv2.namedWindow("YOLO O D", cv2.WINDOW_NORMAL)
cv2.imshow("YOLO O D", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### What you may do ...
1. Run it using different test images
2. Realize its advantages and disadvantages
3. Try different confidence coeeficients/thresholds
4. Realize what is the argmax of the few first detected objects


#### Bye ....